In [ ]:
import sys
from pathlib import Path
import pandas as pd
from lc120_noise import common
from dataclasses import dataclass, asdict
import numpy as np
from typing import List

In [ ]:
def calc_rms_noise(voltages, dt):
    # Remove DC offset
    ac_signal = voltages - np.mean(voltages)
    
    # Integrate power = ∫ v(t)^2 dt
    noise_energy = np.sum(ac_signal**2) * dt
    
    # RMS noise = sqrt(energy / time)
    rms_noise = np.sqrt(noise_energy / (dt * len(voltages)))
    return rms_noise

In [ ]:
@dataclass
class RunLine:
    trace: List[float] # average the traces to a single trace
    current: float
    temp: float
    rms_noise: float

fstructure = common.FileStructure(Path("./data/demo"))
runs = fstructure.get_all_runs()
d = []
    
for run in runs:
    trace, header = run.load_scope_traces()[0]
    dt = float(header[":WAV:XINC?"])
    line = RunLine(trace=trace, temp=run.temp, current=run.current*1000, 
                   rms_noise=calc_rms_noise(trace, dt))
    d.append(asdict(line))
df = pd.DataFrame(d)

In [ ]:
import matplotlib.pyplot as plt

pivot = df.pivot(index='temp', columns='current', values='rms_noise')
fig, ax = plt.subplots(figsize=(10, 6))

# Use imshow or pcolormesh
res = ax.pcolormesh(pivot.columns, pivot.index, pivot.values, shading='auto', cmap='viridis')
fig.colorbar(mappable=res, ax=ax, label='RMS Voltage (V)')
ax.set_xlabel('Current (mA)')
ax.set_ylabel('Temperature (°C)')
ax.set_title('RMS Voltage vs. Current and Temperature')
fig.tight_layout()
fig.savefig("random-data-heat-map.svg")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(df["current"], df["rms_noise"], ',')
ax.set_xlabel('Current (mA)')
ax.set_ylabel('RMS Voltage (V)')
ax.set_title('RMS Voltage vs. Current')
fig.tight_layout()
fig.savefig("current-noise.svg")
plt.show()


fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(df["current"], df["rms_noise"], ',')
ax.set_xlabel('Temperature (°C)')
ax.set_ylabel('RMS Voltage (V)')
ax.set_title('RMS Voltage vs. Temperature')
fig.tight_layout()
fig.savefig("temperature-noise.svg")
plt.show()